In [1]:
import torch
import joblib
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.linear_model import PoissonRegressor
from sklearn.svm import SVR, NuSVR
from scipy.stats import spearmanr, pearsonr

In [2]:
x_train_apsipa = torch.load('../data/X_tensor_APSIPA.pt')
y_train_apsipa = torch.load('../data/y_tensor_APSIPA.pt')
x_train_wpc = torch.load('../data/X_tensor_WPC_cpu.pt')
y_train_wpc = torch.load('../data/y_tensor_WPC.pt')

In [3]:
X_train_apsipa = [[v.detach().numpy() for v in x] for x in x_train_apsipa]
X_train_wpc = [[v.detach().numpy() for v in x] for x in x_train_wpc]

In [4]:
def get_nusvr_model():
    return NuSVR(
    nu=0.42857142857142855,
    kernel='rbf',
    gamma=1, # type: ignore
    degree=2,
    C=50
)

In [5]:
def get_svr_model():
    return SVR(
        kernel='rbf',
        gamma=1,  # type: ignore
        epsilon=1,
        degree=2,
        C=5
    )

In [6]:
def get_pr_model():
    return PoissonRegressor(
        solver='lbfgs',
        max_iter=10,
        fit_intercept=True,
        alpha=0.01
    )

In [7]:
models = ['nusvr', 'svr', 'pr']

In [8]:
results = []

result = {'mode': 'test-in-apsipa'}

nusvr_wpc = get_nusvr_model()
svr_wpc = get_svr_model()
pr_wpc = get_pr_model()

nusvr_wpc.fit(X_train_wpc, y_train_wpc)
svr_wpc.fit(X_train_wpc, y_train_wpc)
pr_wpc.fit(X_train_wpc, y_train_wpc)

ypred_nusvr_wpc = nusvr_wpc.predict(X_train_apsipa)
ypred_svr_wpc = svr_wpc.predict(X_train_apsipa)
ypred_pr_wpc = pr_wpc.predict(X_train_apsipa)

result['nusvr-pearson'] = pearsonr(y_train_apsipa, ypred_nusvr_wpc)[0]
result['nusvr-spearman'] = spearmanr(y_train_apsipa, ypred_nusvr_wpc)[0]
result['svr-pearson'] = pearsonr(y_train_apsipa, ypred_svr_wpc)[0]
result['svr-spearman'] = spearmanr(y_train_apsipa, ypred_svr_wpc)[0]
result['pr-pearson'] = pearsonr(y_train_apsipa, ypred_pr_wpc)[0]
result['pr-spearman'] = spearmanr(y_train_apsipa, ypred_pr_wpc)[0]

results.append(result)

/home/mateusvg_db/TCC/tccVenv2/lib/python3.10/site-packages/sklearn/linear_model/_glm/glm.py:284: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


In [9]:
result = {'mode': 'test-in-wpc'}

nusvr_apsipa = get_nusvr_model()
svr_apsipa = get_svr_model()
pr_apsipa = get_pr_model()

nusvr_apsipa.fit(X_train_apsipa, y_train_apsipa)
svr_apsipa.fit(X_train_apsipa, y_train_apsipa)
pr_apsipa.fit(X_train_apsipa, y_train_apsipa)

ypred_nusvr_apsipa = nusvr_apsipa.predict(X_train_wpc)
ypred_svr_apsipa = svr_apsipa.predict(X_train_wpc)
ypred_pr_apsipa = pr_apsipa.predict(X_train_wpc)

result['nusvr-pearson'] = pearsonr(y_train_wpc, ypred_nusvr_apsipa)[0]
result['nusvr-spearman'] = spearmanr(y_train_wpc, ypred_nusvr_apsipa)[0]
result['svr-pearson'] = pearsonr(y_train_wpc, ypred_svr_apsipa)[0]
result['svr-spearman'] = spearmanr(y_train_wpc, ypred_svr_apsipa)[0]
result['pr-pearson'] = pearsonr(y_train_wpc, ypred_pr_apsipa)[0]
result['pr-spearman'] = spearmanr(y_train_wpc, ypred_pr_apsipa)[0]

results.append(result)

/home/mateusvg_db/TCC/tccVenv2/lib/python3.10/site-packages/sklearn/linear_model/_glm/glm.py:284: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


In [10]:
df_results = pd.DataFrame(results)

In [11]:
df_results

,mode,nusvr-pearson,nusvr-spearman,svr-pearson,svr-spearman,pr-pearson,pr-spearman
0,test-in-apsipa,0.842836,0.923940,0.849672,0.930628,0.868369,0.917857
1,test-in-wpc,0.684894,0.685121,0.735927,0.733120,0.731901,0.733361


In [13]:
joblib.dump(nusvr_wpc, '../data/nusvr_wpc_model.pkl')
joblib.dump(svr_wpc, '../data/svr_wpc_model.pkl')
joblib.dump(pr_wpc, '../data/pr_wpc_model.pkl')

['../data/pr_wpc_model.pkl']